In [1]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [10]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '10-08-2025' \
                    AND div_liquida < '0' \
                    AND pl > 5 \
                    AND pl < 15 \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,ABEV3,"-R$ 14.366.800.000,00","R$ 12,06",12.89,5.87,0.0,0.0,"R$ 14.752.300.000,00","R$ 2.717.720.000,00"
1,PSSA3,"-R$ 9.953.740.000,00","R$ 50,80",10.53,22.40,0.0,0.0,"R$ 3.120.170.000,00","R$ 878.054.000,00"
2,CMIN3,"-R$ 5.099.840.000,00","R$ 4,78",11.81,1.68,0.0,0.0,"R$ 2.220.940.000,00","R$ 115.858.000,00"
3,GRND3,"-R$ 1.323.420.000,00","R$ 5,27",5.86,4.46,0.0,0.0,"R$ 810.837.000,00","R$ 143.575.000,00"
4,LREN3,"-R$ 1.200.190.000,00","R$ 15,44",11.96,9.72,0.0,0.0,"R$ 1.367.960.000,00","R$ 404.496.000,00"
5,CXSE3,"-R$ 1.102.960.000,00","R$ 13,51",9.35,4.44,0.0,0.0,"R$ 4.334.540.000,00","R$ 1.028.410.000,00"
6,FESA3,"-R$ 674.167.000,00","R$ 11,33",14.68,9.54,0.0,0.0,"R$ 272.489.000,00","R$ 18.616.000,00"


In [9]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""
# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
